In [83]:
import pandas as pd


In [84]:
a_list = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

In [85]:
df = pd.DataFrame(a_list[:][0])

In [86]:
df.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [87]:
df.iloc[0]

0         Postcode
1          Borough
2    Neighbourhood
Name: 0, dtype: object

In [88]:
header = df.iloc[0]

In [89]:
df.columns = header

In [90]:
df.head()

,Postcode,Borough,Neighbourhood
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [91]:
df = df.drop(0)

In [92]:
df.head()

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


# 1. Drop rows where Borough is not assigned

In [93]:
df = df[df['Borough'] != "Not assigned"]

In [94]:
df.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


# 2. Combine rows if they have same Postal Code

In [95]:
tdf = df.groupby(['Postcode','Borough'])

In [96]:
tdf = tdf.agg({'Neighbourhood' : ', '.join})

In [97]:
tdf.head()

,,Neighbourhood
Postcode,Borough,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


# 3. If the neighbourhood is not assigned, then the neighborhood will be the same as the borough.

In [98]:
tdf = tdf.reset_index()
tdf.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [102]:
tdf[tdf['Neighbourhood'] == 'Not assigned'] 

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Not assigned


In [103]:
tdf.loc[tdf['Neighbourhood'] == 'Not assigned','Neighbourhood']  = 'Queen\'s Park'

# Extracting Latitude and Longitudes of the Postal codes in the dataframe.

In [115]:
codf  = pd.read_csv('Geospatial_Coordinates.csv')

In [116]:
codf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [117]:
codf.rename(columns={"Postal Code": "Postcode"},inplace = True)

In [118]:
merged = pd.merge(left = tdf, right = codf , on = 'Postcode')

In [119]:
merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [120]:
merged.size

515